In [15]:
# truss_beam_analysis.py

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve

def read_input_file(input_file):
    nodes = []
    elements = []
    boundary_conditions = []
    loads = []
    materials = {}

    with open(input_file, 'r') as f:
        section = None
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                # Identify the current section based on comments.
                section = line[1:].strip().lower()
            elif line:
                if section == 'nodes':
                    node_id, x, y = map(float, line.split())
                    nodes.append((node_id,x,y))
                    # Store node data in an appropriate data structure.
                elif section == 'elements':
                    elem_id, node1, node2, material_id = map(int, line.split())
                    elements.append((node1, node2, material_id))
                elif section == 'materials':
                    material_id, E = map(float, line.split())
                    materials[material_id] = E
                elif section == 'boundary conditions':
                    node_id, dx, dy = map(int, line.split())
                    boundary_conditions.append((node_id, dx, dy))
                elif section == 'loads':
                    node_id, fx, fy = map(int, line.split())
                    loads.append((node_id, fx, fy))

    return nodes, elements, boundary_conditions, loads, materials

def assemble_global_stiffness_matrix(elements):
    num_nodes = max([elem[0] for elem in elements]) + 1
    global_stiffness_matrix = np.zeros((2 * num_nodes, 2 * num_nodes))

    for elem in elements:
        node1, node2, E, _ = elem
        x1, y1 = node1 * 2, node1 * 2 + 1
        x2, y2 = node2 * 2, node2 * 2 + 1

        L = np.sqrt((node2 - node1) ** 2)
        k = (E / L) * np.array([[1, -1], [-1, 1]])

        global_stiffness_matrix[x1:x1 + 2, x1:x1 + 2] += k[:2, :2]
        global_stiffness_matrix[x1:x1 + 2, x2:x2 + 2] += k[:2, 2:]
        global_stiffness_matrix[x2:x2 + 2, x1:x1 + 2] += k[2:, :2]
        global_stiffness_matrix[x2:x2 + 2, x2:x2 + 2] += k[2:, 2:]

    return global_stiffness_matrix

def apply_boundary_conditions(global_stiffness_matrix, loads, boundary_conditions):
    for bc in boundary_conditions:
        node, dx, dy = bc
        idx = node * 2
        if dx:
            global_stiffness_matrix[idx, :] = 0
            global_stiffness_matrix[:, idx] = 0
            global_stiffness_matrix[idx, idx] = 1
        if dy:
            global_stiffness_matrix[idx + 1, :] = 0
            global_stiffness_matrix[:, idx + 1] = 0
            global_stiffness_matrix[idx + 1, idx + 1] = 1

    load_vector = np.zeros(global_stiffness_matrix.shape[0])
    for load in loads:
        node, fx, fy = load
        load_vector[node * 2] = fx
        load_vector[node * 2 + 1] = fy

    return global_stiffness_matrix, load_vector

def solve_displacements(global_stiffness_matrix, load_vector):
    displacements = solve(global_stiffness_matrix, load_vector)
    return displacements

def calculate_element_stresses(elements, displacements):
    element_stresses = []
    for elem in elements:
        node1, node2, E, _ = elem
        x1, y1 = node1 * 2, node1 * 2 + 1
        x2, y2 = node2 * 2, node2 * 2 + 1

        dx = displacements[x2] - displacements[x1]
        dy = displacements[y2] - displacements[y1]

        L = np.sqrt((node2 - node1) ** 2)
        stress = (E / L) * np.array([-dx, -dy, dx, dy])
        element_stresses.append(stress)

    return element_stresses

def plot_deformation(elements, displacements):
    node_coordinates = np.array([
        [0.0, 0.0],
        [2.0, 0.0],
        [4.0, 0.0],
        [0.0, 2.0],
        [2.0, 2.0],
        [4.0, 2.0]
    ])
    
    deformed_coordinates = node_coordinates + displacements.reshape(-1, 2)
    
    plt.figure(figsize=(8, 6))
    plt.title("Deformation of Truss/Beam")
    
    for element in elements:
        node1, node2, _ = element
        x1, y1 = deformed_coordinates[node1]
        x2, y2 = deformed_coordinates[node2]
        plt.plot([x1, x2], [y1, y2], 'b-', linewidth=2)
    
    for node, coords in enumerate(deformed_coordinates):
        plt.plot(coords[0], coords[1], 'ro')
        plt.text(coords[0], coords[1] + 0.1, f'Node {node}', fontsize=12, ha='center')
    
    plt.xlabel("X-axis (meters)")
    plt.ylabel("Y-axis (meters)")
    plt.grid(True)
    plt.axis('equal')
    plt.show()

def plot_stresses(elements, element_stresses):
    plt.figure(figsize=(10, 6))
    plt.title("Stress Distribution in Truss/Beam Elements")
    
    for element, stress in zip(elements, element_stresses):
        node1, node2, _, _ = element
        x1, y1 = node_coordinates[node1]
        x2, y2 = node_coordinates[node2]
        plt.plot([x1, x2], [y1, y2], 'b-', linewidth=2, label=f'Element {element}')
        
        avg_stress = np.mean(np.abs(stress))
        plt.text((x1 + x2) / 2, (y1 + y2) / 2, f'Stress: {avg_stress:.2f} Pa', fontsize=10, ha='center')
    
    plt.xlabel("X-axis (meters)")
    plt.ylabel("Y-axis (meters)")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

if __name__ == "__main__":
    input_file = "input.txt"

    # Step 1: Read input file and create the truss/beam model.
    nodes, elements, boundary_conditions, loads, materials = read_input_file(input_file)
    nodes
    #print(*nodes)
    # Step 2: Assemble the global stiffness matrix.
    '''
    global_stiffness_matrix = assemble_global_stiffness_matrix(elements)

    # Step 3: Apply boundary conditions.
    global_stiffness_matrix, load_vector = apply_boundary_conditions(
        global_stiffness_matrix, loads, boundary_conditions
    )

    # Step 4: Solve for displacements.
    displacements = solve_displacements(global_stiffness_matrix, load_vector)

    # Step 5: Calculate element stresses.
    element_stresses = calculate_element_stresses(elements, displacements)

    # Step 6: Plot deformation and stresses.
    plot_deformation(elements, displacements)
    plot_stresses(elements, element_stresses)
    '''''
